In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-9jbmaud6
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-9jbmaud6
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=c1feef16f2817c9de2a7920530fcff32d705b79b3e2a35098c58c14b332b7eb9
  Stored in directory: /tmp/pip-ephem-wheel-cache-cc3fgeh8/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
##GPU 사용 시
device = torch.device("cuda:0")

In [7]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [8]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2021-05-24 01:51:53--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2021-05-24 01:51:54--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8b528fa138783601dc9dd31749.dl.dropboxusercontent.com/cd/0/get/BPG-ysARhnuLEYxzV4WRSlYcY2rhsSLyLQ4-WQcHOYMMevb5ba_6CdxMMa-FscoVsbII6ZBIL37hhtvvWjAZqDp6DwT7qOdxQGCmoY5_khkBSLFZWWt6-GlIqbkRBvW0nhZ-51w-Y-y0HfPn9UTfSlH_/file?dl=1# [following]
--2021-05-24 01:51:54--  https://uc8b528fa138783601dc9dd31749.dl.dropboxusercontent.com/cd/0/get/BPG-ysARhnuLEYxzV4WRSlYcY2rhsSLyLQ4-WQcHOYMMevb5ba_6CdxMMa-FscoVsbII6ZBI

In [9]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [10]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [12]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [14]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for data in train_dataloader:
  print("Data: ", data)
  print("Waveform: {}\nSample rate: {}\nLabels: {}".format(data[0], data[1], data[2]))
  break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Data:  [tensor([[   2, 3093, 1698,  ...,    1,    1,    1],
        [   2,  517, 7989,  ...,    1,    1,    1],
        [   2, 1458, 7191,  ...,    1,    1,    1],
        ...,
        [   2, 2485, 6821,  ...,    1,    1,    1],
        [   2, 4841, 6333,  ...,    1,    1,    1],
        [   2, 2355, 5842,  ...,    1,    1,    1]], dtype=torch.int32), tensor([15, 36, 14, 26, 47, 41, 16, 64, 13, 38, 17, 35, 29, 34, 34, 13, 45, 23,
        35, 27, 27, 10, 64, 17, 11, 42, 19,  9,  6, 36, 33, 25, 12, 24, 14, 15,
        23, 30, 14, 12, 11, 57, 19, 30, 59, 17, 64, 38, 22, 64, 32, 32, 18, 41,
         4,  5,  3, 64,  8, 11, 14, 23, 26, 56], dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
        0, 0, 1, 0,

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [17]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [18]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [19]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [20]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [21]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.709398627281189 train acc 0.53125
epoch 1 batch id 201 loss 0.5576251745223999 train acc 0.5458644278606966
epoch 1 batch id 401 loss 0.5252853035926819 train acc 0.6546524314214464
epoch 1 batch id 601 loss 0.47527772188186646 train acc 0.709130615640599
epoch 1 batch id 801 loss 0.49085113406181335 train acc 0.74167056803995
epoch 1 batch id 1001 loss 0.28046074509620667 train acc 0.7606924325674326
epoch 1 batch id 1201 loss 0.28258585929870605 train acc 0.7756947335553706
epoch 1 batch id 1401 loss 0.4646575152873993 train acc 0.7861906673804425
epoch 1 batch id 1601 loss 0.3037413954734802 train acc 0.7956062617114303
epoch 1 batch id 1801 loss 0.31250467896461487 train acc 0.8026183370349805
epoch 1 batch id 2001 loss 0.3279321491718292 train acc 0.8095249250374813
epoch 1 batch id 2201 loss 0.2928837537765503 train acc 0.8149207746478874

epoch 1 train acc 0.8187682202787258


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.871843030690537


...


epoch 3 test acc 0.889266304347826


epoch 4 batch id 1 loss 0.3620269000530243 train acc 0.859375
epoch 4 batch id 201 loss 0.0606105662882328 train acc 0.9418532338308457
epoch 4 batch id 401 loss 0.1536085158586502 train acc 0.9432278678304239
epoch 4 batch id 601 loss 0.26867109537124634 train acc 0.9458194675540765
epoch 4 batch id 801 loss 0.2324158251285553 train acc 0.9472534332084894
epoch 4 batch id 1001 loss 0.08410532027482986 train acc 0.9491602147852148
epoch 4 batch id 1201 loss 0.058125823736190796 train acc 0.9513296211490425
epoch 4 batch id 1401 loss 0.17714393138885498 train acc 0.9527681120628123
epoch 4 batch id 1601 loss 0.2428399622440338 train acc 0.9536812929419113
epoch 4 batch id 1801 loss 0.11890430748462677 train acc 0.9551551221543587
epoch 4 batch id 2001 loss 0.2390689104795456 train acc 0.9560688405797102
epoch 4 batch id 2201 loss 0.14855685830116272 train acc 0.9563124716038165

epoch 4 train acc 0.9570334719852105



epoch 4 test acc 0.8912044437340153


epoch 5 batch id 1 loss 0.4134139120578766 train acc 0.859375
epoch 5 batch id 201 loss 0.03861873596906662 train acc 0.9633084577114428
epoch 5 batch id 401 loss 0.1580589860677719 train acc 0.9652821072319202
epoch 5 batch id 601 loss 0.1801963746547699 train acc 0.9665661397670549
epoch 5 batch id 801 loss 0.1394248753786087 train acc 0.9674040262172284
epoch 5 batch id 1001 loss 0.08941760659217834 train acc 0.968390984015984
epoch 5 batch id 1201 loss 0.0397331528365612 train acc 0.9694004995836802
epoch 5 batch id 1401 loss 0.11002505570650101 train acc 0.9697425945753033
epoch 5 batch id 1601 loss 0.0815429762005806 train acc 0.9699699406620862
epoch 5 batch id 1801 loss 0.037037480622529984 train acc 0.9706152831760133
epoch 5 batch id 2001 loss 0.16322050988674164 train acc 0.9708661294352824
epoch 5 batch id 2201 loss 0.08305955678224564 train acc 0.9708300204452521

epoch 5 train acc 0.9711341901308305



epoch 5 test acc 0.891184462915601
